## Imports & chemins

In [96]:
# ===== Projet Immobilier — Chargement, Nettoyage, Codes Postaux (robuste) + Dashboard =====
import os, pandas as pd, numpy as np
from pathlib import Path

RAW_DIR   = os.path.abspath(os.path.join("..", "data", "raw"))
CLEAN_DIR = os.path.abspath(os.path.join("..", "data", "clean"))
Path(CLEAN_DIR).mkdir(parents=True, exist_ok=True)

def to_num_fr(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.replace("\u00A0","", regex=False)
    s = s.str.replace(" ", "", regex=False).str.replace(",", ".", regex=False)
    return pd.to_numeric(s, errors="coerce")

def parse_dates_robust(s: pd.Series) -> pd.Series:
    x = (s.astype(str).str.replace("\u00A0"," ", regex=False)
                 .str.strip().str.replace(r"[^0-9/\-]", "", regex=True))
    out = pd.to_datetime(x, errors="coerce", dayfirst=True)
    mask = out.isna()
    if mask.any():
        out.loc[mask] = pd.to_datetime(x.loc[mask], errors="coerce", format="%Y-%m-%d")
    return out

def load_dvf_idf(dvf_path: str, out_fp: str) -> pd.DataFrame:
    if os.path.exists(out_fp):
        return pd.read_parquet(out_fp)

    usecols = ["Date mutation","Nature mutation","Valeur fonciere",
               "Code postal","Commune","Code departement","Code commune",
               "Type local","Surface reelle bati","Nombre pieces principales"]
    idf = ("75","77","78","91","92","93","94","95")

    chunks, tried = [], []
    for enc in ("cp1252","latin1","utf-8","utf-8-sig"):
        try:
            for chunk in pd.read_csv(dvf_path, sep="|", usecols=usecols,
                                     encoding=enc, dtype=str, chunksize=200_000):
                # Renommage
                chunk = chunk.rename(columns={
                    "Date mutation":"date_mutation",
                    "Valeur fonciere":"valeur_fonciere",
                    "Surface reelle bati":"surface_reelle_bati",
                    "Commune":"nom_commune",
                    "Code postal":"code_postal",
                    "Type local":"type_local",
                    "Code departement":"code_departement",
                    "Code commune":"code_commune_3",
                })

                # Île-de-France
                m_idf = chunk["code_departement"].astype(str).str.strip().str[:2].isin(idf)
                chunk = chunk[m_idf]

                # Types et conversions
                chunk["date_mutation"]       = parse_dates_robust(chunk["date_mutation"])
                chunk["annee"]               = chunk["date_mutation"].dt.year
                chunk["valeur_fonciere"]     = to_num_fr(chunk["valeur_fonciere"])
                chunk["surface_reelle_bati"] = to_num_fr(chunk["surface_reelle_bati"])

                # Codes & prix/m²
                chunk["code_departement"]     = chunk["code_departement"].astype(str).str.strip().str.zfill(2)
                chunk["code_commune_3"]       = chunk["code_commune_3"].astype(str).str.strip().str.zfill(3)
                chunk["code_commune_insee"]   = chunk["code_departement"] + chunk["code_commune_3"]
                chunk["prix_m2"]              = chunk["valeur_fonciere"] / chunk["surface_reelle_bati"]

                # Sélection minimale
                keep = chunk[[
                    "date_mutation","annee","valeur_fonciere","surface_reelle_bati","prix_m2",
                    "nom_commune","code_commune_insee","code_postal"
                ]]

                chunks.append(keep)
            break
        except Exception as e:
            tried.append((enc, str(e)))
            continue

    if not chunks:
        raise RuntimeError(f"Lecture DVF vide. Encodings testés: {tried}")

    df = pd.concat(chunks, ignore_index=True)

    # Filtres de cohérence
    df = df.dropna(subset=["date_mutation","annee","valeur_fonciere","surface_reelle_bati","prix_m2","nom_commune"])
    df = df[(df["surface_reelle_bati"] > 8) & (df["valeur_fonciere"] > 1000) & (df["prix_m2"].between(100, 30000))]
    df.to_parquet(out_fp, index=False)
    return df

# --- Chargement DVF ---
dvf_path = os.path.join(RAW_DIR, "DVF_2025_S1.txt")
clean_fp = os.path.join(CLEAN_DIR, "dvf_clean.parquet")
dvf = load_dvf_idf(dvf_path, clean_fp)

# --- Jointure Codes Postaux (hyper-robuste encodage + noms/structures variés) ---
import re

codes_csv = os.path.join(RAW_DIR, "Base_codes_postaux.csv")

def _norm_colname(c: str) -> str:
    return re.sub(r"[^a-z0-9]+", "_", c.lower()).strip("_")

def read_codes_postaux(path: str) -> pd.DataFrame:
    # 1) Lecture robuste (encodage + séparateur auto)
    encodings = ["cp1252", "latin1", "utf-8", "utf-8-sig"]
    last_err = None
    for enc in encodings:
        try:
            dfc = pd.read_csv(path, sep=None, engine="python", encoding=enc, dtype=str)
            break
        except Exception as e:
            last_err = e
    else:
        raise last_err

    # 2) Normalisation des noms de colonnes
    orig_cols = list(dfc.columns)
    norm_cols = [_norm_colname(c) for c in dfc.columns]
    dfc.columns = norm_cols

    # 3) Alias possibles
    insee_aliases   = {
        "code_commune_insee","code_insee","insee","insee_code",
        "codecommuneinsee","codgeo","code_commune_insee_2020","code_commune_insee_2024"
    }
    postal_aliases  = {
        "code_postal","cp","postal","postal_code","codepostal","code_postal_commune"
    }
    dept_aliases    = {"code_departement","departement","dep","code_dept","dept","code_dep"}
    com3_aliases    = {"code_commune","codecom","codcom","code_comm","code_commune_3"}

    # 4) Détection colonnes
    cols_set = set(dfc.columns)

    def pick_any(candidates:set[str]) -> str|None:
        for c in candidates:
            if c in cols_set:
                return c
        return None

    col_insee  = pick_any(insee_aliases)
    col_cp     = pick_any(postal_aliases)
    col_dept   = pick_any(dept_aliases)
    col_com3   = pick_any(com3_aliases)

    # 5) Si INSEE manquant, mais dept + com3 dispos -> construire
    if col_insee is None and (col_dept is not None and col_com3 is not None):
        # sécuriser formats
        dfc[col_dept] = dfc[col_dept].astype(str).str.strip().str.zfill(2)
        dfc[col_com3] = dfc[col_com3].astype(str).str.strip().str.zfill(3)
        dfc["code_commune_insee"] = dfc[col_dept] + dfc[col_com3]
        col_insee = "code_commune_insee"

    # 6) Si CP introuvable mais colonne nom proche "code_postal" existe avec typos courantes
    if col_cp is None:
        # Heuristique: chercher une colonne qui contient "cp" ou "postal"
        candidates = [c for c in dfc.columns if ("postal" in c or c == "cp" or c.endswith("_cp"))]
        col_cp = candidates[0] if candidates else None

    # 7) Validation
    if col_insee is None or col_cp is None:
        # Aide au debug: montrer colonnes disponibles
        raise ValueError(
            "Le fichier codes postaux doit contenir INSEE et CP.\n"
            f"Colonnes disponibles (normalisées): {dfc.columns.tolist()}\n"
            "Essayé: INSEE in {insee_aliases} ou (dept in {dept_aliases} + com3 in {com3_aliases}); "
            f"CP in {postal_aliases}."
        )

    # 8) Sélection + nettoyage minimal
    out = dfc[[col_insee, col_cp]].rename(columns={col_insee: "code_commune_insee", col_cp: "code_postal"}).copy()
    out["code_commune_insee"] = out["code_commune_insee"].astype(str).str.strip()
    out["code_postal"]        = out["code_postal"].astype(str).str.strip()
    out = out.dropna(subset=["code_commune_insee","code_postal"])
    # garder CP à 5 chiffres si applicable
    out["code_postal"] = out["code_postal"].str.extract(r"(\d{5})", expand=False).fillna(out["code_postal"])
    return out

codes = read_codes_postaux(codes_csv)

# Choix d’un CP de référence par INSEE (mode sinon 1er)
cp_ref = (codes.groupby("code_commune_insee")["code_postal"]
               .agg(lambda s: s.mode().iat[0] if not s.mode().empty else s.iloc[0])
               .rename("code_postal_ref")
               .reset_index())

# Merge sécurisé
dvf = dvf.drop(columns=["code_postal"], errors="ignore").merge(cp_ref, on="code_commune_insee", how="left")
dvf = dvf.rename(columns={"code_postal_ref":"code_postal"})
dvf.to_parquet(clean_fp, index=False)


# ================================== Dashboard interactif ==================================
import ipywidgets as W
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

df = pd.read_parquet(clean_fp).copy()

def iqr_bounds(s, k=2.0):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    return q1 - k*iqr, q3 + k*iqr

def apply_outliers(d, use_iqr=True):
    if (not use_iqr) or (len(d) < 50):
        return d
    lo, hi = iqr_bounds(d["prix_m2"], k=2.0)
    return d[d["prix_m2"].between(lo, hi)]

def filter_by_surface(d, s_range):
    smin, smax = s_range
    return d[d["surface_reelle_bati"].between(smin, smax)]

def compute_yield(d, loyer_m2):
    out = d.copy()
    out["revenu_annuel"] = loyer_m2 * out["surface_reelle_bati"] * 12.0
    out["yield_brut"]    = out["revenu_annuel"] / out["valeur_fonciere"]
    return out

# Widgets
w_surface = W.IntRangeSlider(value=(20, 80), min=10, max=200, step=1,
                             description="Surface (m²)", continuous_update=False)
w_loyer   = W.FloatSlider(value=22.0, min=5.0, max=45.0, step=0.5,
                          readout_format=".1f", description="Loyer €/m²", continuous_update=False)
w_topn    = W.IntSlider(value=15, min=5, max=50, step=1, description="Top N", continuous_update=False)
w_out     = W.Checkbox(value=True, description="Filtrer outliers (IQR)")
w_show_hist = W.Checkbox(value=True, description="Histogramme prix/m²")

out = W.Output()

def render(_=None):
    with out:
        clear_output(wait=True)
        d = filter_by_surface(df, w_surface.value)
        d = apply_outliers(d, w_out.value)
        d = compute_yield(d, w_loyer.value)

        nb     = len(d)
        p50    = float(d["prix_m2"].median()) if nb else np.nan
        y_med  = float(d["yield_brut"].median()) if nb else np.nan
        html = f"""
        <div>
          <h3>KPIs</h3>
          <ul>
            <li>Transactions : {nb:,}</li>
            <li>Prix/m² (médiane) : {p50:,.0f} €</li>
            <li>Rendement brut (médiane) : {y_med*100:,.1f} %</li>
          </ul>
        </div>
        """
        display(W.HTML(html))

        if nb:
            # Top N communes par rendement médian
            top = (d.groupby("nom_commune", as_index=False)["yield_brut"]
                     .median().sort_values("yield_brut", ascending=False).head(w_topn.value))
            display(top)

            # Scatter prix/m² vs surface
            plt.figure(figsize=(6,4))
            d.sample(min(2000, nb), random_state=0).plot(kind="scatter", x="surface_reelle_bati", y="prix_m2")
            plt.title("Prix/m² vs Surface (échantillon)")
            plt.xlabel("Surface (m²)"); plt.ylabel("Prix/m² (€)")
            plt.show()

            # Histogramme prix/m² (optionnel)
            if w_show_hist.value:
                plt.figure(figsize=(6,4))
                d["prix_m2"].dropna().clip(0, d["prix_m2"].quantile(0.99)).plot(kind="hist", bins=40)
                plt.title("Distribution du prix/m² (troncature au 99e centile)")
                plt.xlabel("Prix/m² (€)")
                plt.ylabel("Fréquence")
                plt.show()

controls = W.VBox([
    W.HTML("<h3>Paramètres d’analyse</h3>"),
    W.HBox([w_surface, w_loyer]),
    W.HBox([w_topn, w_out, w_show_hist])
])

ui = W.HBox([controls, out], layout=W.Layout(align_items="flex-start"))
display(ui)

for w in (w_surface, w_loyer, w_topn, w_out, w_show_hist):
    w.observe(render, "value")

render()


In [ ]:
# ============================= Dashboard esthétique (widgets + UI) =============================
import ipywidgets as W
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, HTML

# -- Thème & helpers UI -------------------------------------------------------------------------
CSS = """
<style>
:root{
  --bg:#0f172a;         /* slate-900 */
  --panel:#111827;      /* gray-900 */
  --card:#131a2a;       /* custom */
  --muted:#9ca3af;      /* gray-400 */
  --text:#e5e7eb;       /* gray-200 */
  --accent:#22d3ee;     /* cyan-400 */
  --accent2:#60a5fa;    /* blue-400 */
}
div.app-wrap{
  background:linear-gradient(135deg,#0f172a 0%,#0b1220 100%);
  color:var(--text);
  padding:16px 16px 24px 16px; border-radius:16px;
  box-shadow:0 10px 30px rgba(0,0,0,.35);
  font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, Cantarell, Noto Sans, Helvetica Neue, Arial;
}
.app-title{
  font-size:20px; font-weight:600; letter-spacing:.2px; margin:0 0 8px 0;
}
.app-subtitle{
  color:var(--muted); font-size:13px; margin:0 0 14px 0;
}
.grid-2{
  display:grid; grid-template-columns: 1.1fr 1fr; gap:16px; align-items:start;
}
.card{
  background:var(--panel); border:1px solid rgba(255,255,255,.06);
  border-radius:14px; padding:12px 12px;
}
.controls-grid{
  display:grid; grid-template-columns:1fr 1fr; gap:10px;
}
.kpis{ display:grid; grid-template-columns: repeat(3, 1fr); gap:10px; }
.kpi{
  background:var(--card); border:1px solid rgba(255,255,255,.06);
  border-radius:12px; padding:12px;
}
.kpi .label{ color:var(--muted); font-size:12px; }
.kpi .value{ font-size:22px; font-weight:600; color:var(--accent); }
.tbl-title{ font-size:14px; font-weight:600; margin:12px 0 6px 0; }
.footer-note{ color:var(--muted); font-size:12px; margin-top:8px; }
</style>
"""

def fmt_int(x):
    return f"{int(x):,}".replace(",", " ")

def kpi_card(label, value_html):
    return f"""
    <div class="kpi">
      <div class="label">{label}</div>
      <div class="value">{value_html}</div>
    </div>
    """

display(HTML(CSS + '<div class="app-wrap"><div class="app-title">Analyse DVF — Île-de-France</div><div class="app-subtitle">Nettoyage, filtres interactifs & exploration</div></div>'))

# -- Données -------------------------------------------------------------------------------------
df = pd.read_parquet(clean_fp).copy()
# sécurité colonnes
for c in ["prix_m2","surface_reelle_bati","valeur_fonciere", "annee", "nom_commune", "code_postal"]:
    if c not in df.columns:
        df[c] = np.nan

# -- Widgets -------------------------------------------------------------------------------------
w_surface = W.IntRangeSlider(value=(20, 80), min=10, max=200, step=1,
                             description="Surface (m²)", continuous_update=False, readout=True,
                             style={'description_width':'110px'}, layout=W.Layout(width='100%'))
w_loyer   = W.FloatSlider(value=22.0, min=5.0, max=45.0, step=0.5,
                          readout_format=".1f", description="Loyer €/m²", continuous_update=False,
                          style={'description_width':'110px'}, layout=W.Layout(width='100%'))
w_topn    = W.IntSlider(value=15, min=5, max=50, step=1,
                        description="Top N", continuous_update=False,
                        style={'description_width':'110px'}, layout=W.Layout(width='100%'))
w_iqr     = W.ToggleButtons(options=[('IQR on', True), ('IQR off', False)],
                            value=True, description="Outliers",
                            style={'description_width':'110px'})
w_hist    = W.Checkbox(value=True, description="Histogramme prix/m²")
w_year    = W.SelectionRangeSlider(
              options=sorted(df["annee"].dropna().astype(int).unique()),
              index=(0, max(0, len(df["annee"].dropna().unique())-1)),
              description="Années", continuous_update=False,
              layout=W.Layout(width='100%'),
              style={'description_width':'110px'}
           ) if df["annee"].notna().any() else W.Label("Années : n/a")

communes = ["(Toutes)"] + sorted(df["nom_commune"].dropna().astype(str).unique().tolist())
w_commune = W.Dropdown(options=communes, value="(Toutes)", description="Commune",
                       layout=W.Layout(width='100%'), style={'description_width':'110px'})

w_btn_export = W.Button(description="Exporter (CSV)", button_style='',
                        tooltip="Exporte le dataset filtré en CSV dans le dossier clean/",
                        layout=W.Layout(width='160px'))
w_msg = W.HTML("")

# -- Sorties -------------------------------------------------------------------------------------
out_overview = W.Output()
out_table    = W.Output()
out_plot1    = W.Output()
out_plot2    = W.Output()

# -- Filtres & calculs ---------------------------------------------------------------------------
def iqr_bounds(s, k=2.0):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    return q1 - k*iqr, q3 + k*iqr

def apply_filters(d):
    # surface
    smin, smax = w_surface.value
    d = d[d["surface_reelle_bati"].between(smin, smax)]
    # années
    if isinstance(w_year, W.SelectionRangeSlider):
        y0, y1 = w_year.value
        d = d[(d["annee"]>=y0) & (d["annee"]<=y1)]
    # commune
    if w_commune.value != "(Toutes)":
        d = d[d["nom_commune"] == w_commune.value]
    # outliers
    if w_iqr.value and len(d) >= 50 and d["prix_m2"].notna().any():
        lo, hi = iqr_bounds(d["prix_m2"], k=2.0)
        d = d[d["prix_m2"].between(lo, hi)]
    # yield
    loyer_m2 = w_loyer.value
    d = d.assign(revenu_annuel = loyer_m2 * d["surface_reelle_bati"] * 12.0)
    d = d.assign(yield_brut = d["revenu_annuel"] / d["valeur_fonciere"])
    return d

# -- Rendu ---------------------------------------------------------------------------------------
def render(_=None):
    with out_overview:
        clear_output(wait=True)
        d = apply_filters(df)

        nb = len(d)
        p50 = float(d["prix_m2"].median()) if d["prix_m2"].notna().any() else np.nan
        ymd = float(d["yield_brut"].median()) if d["yield_brut"].notna().any() else np.nan
        s50 = float(d["surface_reelle_bati"].median()) if d["surface_reelle_bati"].notna().any() else np.nan

        kpis_html = '<div class="kpis">' + \
            kpi_card("Transactions", fmt_int(nb) if nb==nb else "–") + \
            kpi_card("Prix/m² médiane", f"{p50:,.0f} €".replace(",", " ") if p50==p50 else "–") + \
            kpi_card("Rendement brut médian", f"{ymd*100:,.1f} %".replace(",", " ") if ymd==ymd else "–") + \
        '</div>'

        display(HTML(f'<div class="card">{kpis_html}<div class="footer-note">Filtrage interactif à gauche. Export CSV disponible.</div></div>'))

    with out_table:
        clear_output(wait=True)
        d = apply_filters(df)
        # Top N communes par rendement médian
        if len(d):
            top = (d.groupby(["nom_commune","code_postal"], as_index=False)["yield_brut"]
                     .median().sort_values("yield_brut", ascending=False).head(w_topn.value))
            top["yield_brut_%"] = (top["yield_brut"]*100).round(2)
            to_show = top[["nom_commune","code_postal","yield_brut_%"]].rename(
                columns={"nom_commune":"Commune","code_postal":"CP","yield_brut_%":"Yield brut (%)"}
            )
            html = to_show.style.hide(axis="index") \
                               .set_table_styles([{'selector':'th','props':'text-align:left; padding:6px 8px;'},
                                                  {'selector':'td','props':'padding:6px 8px;'}]) \
                               .bar(subset=["Yield brut (%)"], vmin=to_show["Yield brut (%)"].min(),
                                    vmax=to_show["Yield brut (%)"].max()) \
                               .to_html()
            display(HTML('<div class="tbl-title">Top communes (médiane du rendement)</div>'))
            display(HTML(f'<div class="card">{html}</div>'))
        else:
            display(HTML('<div class="card">Aucune donnée avec ces filtres.</div>'))

    # Graphes: Plotly si possible, sinon Matplotlib
    try:
        import plotly.express as px
        with out_plot1:
            clear_output(wait=True)
            d = apply_filters(df)
            if len(d):
                ds = d.sample(min(4000, len(d)), random_state=0)
                fig = px.scatter(ds, x="surface_reelle_bati", y="prix_m2",
                                 hover_data=["nom_commune","code_postal","annee"],
                                 trendline="lowess", height=420)
                fig.update_layout(margin=dict(l=10,r=10,t=10,b=10), template="plotly_dark")
                fig.show()
            else:
                display(HTML('<div class="card">—</div>'))

        with out_plot2:
            clear_output(wait=True)
            d = apply_filters(df)
            if len(d) and w_hist.value and d["prix_m2"].notna().any():
                cut = d["prix_m2"].clip(upper=d["prix_m2"].quantile(0.99))
                fig = px.histogram(cut, x=cut, nbins=45, height=380)
                fig.update_layout(margin=dict(l=10,r=10,t=10,b=10), template="plotly_dark")
                fig.update_xaxes(title="Prix/m² (€)")
                fig.update_yaxes(title="Fréquence")
                fig.show()
            else:
                display(HTML('<div class="card">—</div>'))

    except Exception:
        with out_plot1:
            clear_output(wait=True)
            d = apply_filters(df)
            if len(d):
                ds = d.sample(min(4000, len(d)), random_state=0)
                plt.figure(figsize=(6.8,4.2))
                plt.scatter(ds["surface_reelle_bati"], ds["prix_m2"], s=12, alpha=.6)
                plt.title("Prix/m² vs Surface (échantillon)")
                plt.xlabel("Surface (m²)"); plt.ylabel("Prix/m² (€)")
                plt.grid(alpha=.2); plt.tight_layout()
                plt.show()
            else:
                display(HTML('<div class="card">—</div>'))

        with out_plot2:
            clear_output(wait=True)
            d = apply_filters(df)
            if len(d) and w_hist.value and d["prix_m2"].notna().any():
                cut = d["prix_m2"].clip(upper=d["prix_m2"].quantile(0.99))
                plt.figure(figsize=(6.8,3.8))
                plt.hist(cut, bins=45)
                plt.title("Distribution du prix/m² (troncature 99e centile)")
                plt.xlabel("Prix/m² (€)"); plt.ylabel("Fréquence")
                plt.grid(alpha=.2); plt.tight_layout()
                plt.show()
            else:
                display(HTML('<div class="card">—</div>'))

# -- Export CSV ----------------------------------------------------------------------------------
def on_export_clicked(_):
    d = apply_filters(df)
    out_fp = os.path.join(CLEAN_DIR, "dvf_filtre_export.csv")
    d.to_csv(out_fp, index=False)
    w_msg.value = f'<span style="color:#22d3ee">Exporté : {out_fp}</span>'

w_btn_export.on_click(on_export_clicked)

# -- Mise en page (onglets) ----------------------------------------------------------------------
controls_left = W.VBox([
    W.HTML('<div class="card"><div class="controls-grid"></div></div>'),  # placeholder css card
    W.VBox([
        W.HTML('<div class="card"><div class="app-subtitle" style="margin:0 0 10px 0;">Paramètres</div>'),
        W.VBox([
            w_surface, w_loyer, w_topn, w_iqr, w_hist, w_year, w_commune,
            W.HBox([w_btn_export, w_msg])
        ], layout=W.Layout(padding="0 6px 8px 6px"))
    ])
])

tab = W.Tab(children=[
    W.VBox([out_overview]),
    W.VBox([out_table]),
    W.VBox([out_plot1]),
    W.VBox([out_plot2]),
])
tab.set_title(0, "Vue d’ensemble")
tab.set_title(1, "Communes")
tab.set_title(2, "Dispersion")
tab.set_title(3, "Distribution")

layout = W.HBox([
    W.HTML('<div class="app-wrap" style="width:16px;visibility:hidden"></div>'),
    W.VBox([controls_left], layout=W.Layout(min_width="360px", width="380px")),
    W.VBox([tab], layout=W.Layout(flex="1 1 auto", width="100%"))
])
display(layout)

# -- Observers -----------------------------------------------------------------------------------
for w in (w_surface, w_loyer, w_topn, w_iqr, w_hist, w_commune):
    w.observe(render, "value")
if isinstance(w_year, W.SelectionRangeSlider):
    w_year.observe(render, "value")

render()


## Chargement + typage + IDF + prix/m²

In [81]:
import os, re, unicodedata as ud
import pandas as pd

CLEAN_DIR = os.path.abspath(os.path.join("..", "data", "clean"))
os.makedirs(CLEAN_DIR, exist_ok=True)
codes_parquet = os.path.join(CLEAN_DIR, "codes_postaux.parquet")

def _norm_name(c: str) -> str:
    # normalise: strip, retire '#', enlève accents, lower, remplace non-alnum par '_', compresse '__'
    c = (c or "").strip().lstrip("#")
    c = ud.normalize("NFKD", c).encode("ascii","ignore").decode("ascii")
    c = re.sub(r"[^A-Za-z0-9]+", "_", c).strip("_").lower()
    c = re.sub(r"_+", "_", c)
    return c

def _find_col(cols, candidates):
    """retourne le premier nom présent parmi les 'candidates' (déjà normalisés)"""
    for cand in candidates:
        if cand in cols:
            return cand
    return None

def read_codes_postaux_robust(path, cache_fp=codes_parquet, force_reload=False):
    # 0) Cache parquet ultra-rapide
    if os.path.exists(cache_fp) and not force_reload:
        dfp = pd.read_parquet(cache_fp)
        # vérifie qu'on a bien l'INSEE
        if "code_commune_insee" in dfp.columns:
            return dfp

    # 1) Lecture rapide (moteur C) – encodage cp1252 (classique), sinon latin-1
    try:
        raw = pd.read_csv(path, sep=";", dtype=str, encoding="cp1252", engine="c", low_memory=True)
    except UnicodeDecodeError:
        raw = pd.read_csv(path, sep=";", dtype=str, encoding="latin-1", engine="c", low_memory=True)

    # 2) Normalise tous les noms de colonnes
    norm_cols = [_norm_name(c) for c in raw.columns]
    raw.columns = norm_cols
    cols = set(norm_cols)

    # 3) Détection robuste des colonnes utiles
    insee_col = _find_col(cols, ["code_commune_insee", "code_insee_commune", "codeinsee", "insee", "code_commune"])
    cp_col    = _find_col(cols, ["code_postal", "cp", "codepostal"])
    nom_col   = _find_col(cols, ["nom_de_la_commune", "nom_commune", "commune", "nom"])
    lib_col   = _find_col(cols, ["libelle_d_acheminement", "libelle_acheminement", "libelle_d_acheminement"])
    l5_col    = _find_col(cols, ["ligne_5", "ligne5"])

    if insee_col is None:
        # aide au debug : affiche les colonnes rencontrées
        raise KeyError(
            "Impossible de trouver la colonne INSEE. Colonnes normalisées lues : "
            + ", ".join(sorted(cols))
        )

    # 4) Sous-sélection & renommage stable
    keep_map = {}
    keep_map[insee_col] = "code_commune_insee"
    if cp_col:  keep_map[cp_col]  = "code_postal"
    if nom_col: keep_map[nom_col] = "nom_commune"
    if lib_col: keep_map[lib_col] = "libelle_acheminement"
    if l5_col:  keep_map[l5_col]  = "ligne_5"

    df = raw[list(keep_map.keys())].rename(columns=keep_map)

    # 5) Formats
    df["code_commune_insee"] = df["code_commune_insee"].astype(str).str.strip().str.zfill(5)
    if "code_postal" in df.columns:
        df["code_postal"] = df["code_postal"].astype(str).str.strip().str.zfill(5)

    # 6) Réduction à 1 ligne par INSEE (code postal “référence” choisi par ordre)
    # 6) Réduction à 1 ligne par INSEE (choix d’un code postal “référence” par ordre)
    agg_dict = {}
    if "code_postal" in df.columns:
        agg_dict["code_postal"] = "first"
    if "nom_commune" in df.columns:
        agg_dict["nom_commune"] = "first"
    if "libelle_acheminement" in df.columns:
        agg_dict["libelle_acheminement"] = "first"

    # Si aucune colonne à agréger n'est dispo, on garde juste la liste unique des INSEE
    if not agg_dict:
        df_ref = df[["code_commune_insee"]].drop_duplicates().copy()
    else:
        sort_cols = ["code_commune_insee"] + (["code_postal"] if "code_postal" in df.columns else [])
        df_ref = (
            df.sort_values(sort_cols)
            .groupby("code_commune_insee", as_index=False)
            .agg(agg_dict)                # <-- dict, pas **kwargs
        )

    df_ref.to_parquet(cache_fp, index=False)
    return df_ref
codes = read_codes_postaux_robust(codes_csv, force_reload=True)
print("CODES ->", codes.shape, list(codes.columns))
codes.head()



CODES -> (35007, 4) ['code_commune_insee', 'code_postal', 'nom_commune', 'libelle_acheminement']


,code_commune_insee,code_postal,nom_commune,libelle_acheminement
0,01001,01400,L ABERGEMENT CLEMENCIAT,L ABERGEMENT CLEMENCIAT
1,01002,01640,L ABERGEMENT DE VAREY,L ABERGEMENT DE VAREY
2,01004,01500,AMBERIEU EN BUGEY,AMBERIEU EN BUGEY
3,01005,01330,AMBERIEUX EN DOMBES,AMBERIEUX EN DOMBES
4,01006,01300,AMBLEON,AMBLEON


## Lecture de la base codes postaux

In [82]:
def load_dvf_chunked(dvf_path, clean_fp):
    # Si un parquet propre existe déjà : le lire directement (plus rapide)
    if os.path.exists(clean_fp):
        df = pd.read_parquet(clean_fp)
        return df

    usecols = [
        "Date mutation","Nature mutation","Valeur fonciere",
        "Code postal","Commune","Code departement","Code commune",
        "Type local","Surface reelle bati","Nombre pieces principales"
    ]
    idf_prefix = ("75","77","78","91","92","93","94","95")

    keep = []
    chunksize = 200_000
    # encodage DVF souvent cp1252
    for i, chunk in enumerate(pd.read_csv(
        dvf_path, sep="|", dtype=str, usecols=usecols,
        chunksize=chunksize, low_memory=True, engine="c", encoding="cp1252"
    )):
        # vente + IDF
        chunk = chunk[chunk["Nature mutation"].fillna("").str.contains("Vente", case=False, na=False)]
        chunk = chunk[chunk["Code departement"].astype(str).str.startswith(idf_prefix)]
        if chunk.empty:
            continue

        # num + filtres
        chunk["Valeur fonciere"] = to_num_fr(chunk["Valeur fonciere"])
        chunk["Surface reelle bati"] = pd.to_numeric(chunk["Surface reelle bati"], errors="coerce")

        chunk = chunk[(chunk["Surface reelle bati"] > 8) & (chunk["Valeur fonciere"] > 1000)]
        chunk["prix_m2"] = chunk["Valeur fonciere"] / chunk["Surface reelle bati"]
        chunk = chunk[chunk["prix_m2"].between(100, 30000)]

        # dates + renommage
        chunk["Date mutation"] = pd.to_datetime(chunk["Date mutation"], errors="coerce")
        chunk = chunk.rename(columns={
            "Date mutation":"date_mutation",
            "Valeur fonciere":"valeur_fonciere",
            "Surface reelle bati":"surface_reelle_bati",
            "Commune":"nom_commune",
            "Code postal":"code_postal",
            "Type local":"type_local",
            "Code departement":"code_departement",
            "Code commune":"code_commune_3"
        })

        # construire code INSEE (dept 2 + commune 3) — valable en IDF
        chunk["code_departement"] = chunk["code_departement"].astype(str).str.strip().str.zfill(2)
        chunk["code_commune_3"]  = chunk["code_commune_3"].astype(str).str.strip().str.zfill(3)
        chunk["code_commune_insee"] = chunk["code_departement"] + chunk["code_commune_3"]

        keep.append(chunk[[
            "date_mutation","valeur_fonciere","surface_reelle_bati","prix_m2",
            "nom_commune","code_postal","type_local","code_commune_insee"
        ]])

        if (i+1) % 5 == 0:
            print(f"[Progression] {i+1} chunks traités...")

    if not keep:
        raise RuntimeError("Aucune ligne DVF retenue après filtrage.")

    df = pd.concat(keep, ignore_index=True)
    df["annee"] = pd.to_datetime(df["date_mutation"], errors="coerce").dt.year

    # Merge codes postaux via INSEE → 1 ligne par INSEE côté référentiel
    codes_ref = (codes
                 .dropna(subset=["code_commune_insee"])
                 .sort_values(["code_commune_insee","code_postal"])
                 .groupby("code_commune_insee", as_index=False)
                 .agg(code_postal_ref=("code_postal","first"),
                      nom_commune_ref=("nom_commune","first"),
                      libelle_acheminement_ref=("libelle_acheminement","first")))

    df = df.merge(codes_ref, on="code_commune_insee", how="left")
    # harmoniser nom_commune si manquants
    if "nom_commune" not in df.columns or df["nom_commune"].isna().mean() > 0.5:
        df["nom_commune"] = df["nom_commune_ref"].fillna(df.get("nom_commune", pd.Series(index=df.index)))

    # colonnes finales minimales
    df_min = df[[
        "date_mutation","annee","valeur_fonciere","surface_reelle_bati","prix_m2",
        "nom_commune","code_commune_insee","code_postal_ref"
    ]].rename(columns={"code_postal_ref":"code_postal"})

    df_min.to_parquet(clean_fp, index=False)
    print(f"[OK] Sauvegardé → {clean_fp} ({len(df_min):,} lignes)")
    return df_min

clean_fp = os.path.join(CLEAN_DIR, "dvf_clean.parquet")
dvf = load_dvf_chunked(dvf_txt, clean_fp)
print("DVF ->", dvf.shape, list(dvf.columns))


DVF -> (56612, 9) ['date_mutation', 'valeur_fonciere', 'surface_reelle_bati', 'prix_m2', 'nom_commune', 'code_postal', 'type_local', 'code_commune_insee', 'annee']


In [83]:
import os
clean_fp = os.path.join("..", "data", "clean", "dvf_clean.parquet")
if os.path.exists(clean_fp):
    os.remove(clean_fp)
    print("Deleted:", clean_fp)
import os, pandas as pd

RAW_DIR   = os.path.abspath(os.path.join("..", "data", "raw"))
CLEAN_DIR = os.path.abspath(os.path.join("..", "data", "clean"))
os.makedirs(CLEAN_DIR, exist_ok=True)

raw_txt  = os.path.join(RAW_DIR, "DVF_2025_S1.txt")
clean_fp = os.path.join(CLEAN_DIR, "dvf_clean.parquet")

def to_num_fr(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.replace("\u00A0","", regex=False)  # NBSP
    s = s.str.replace(" ", "", regex=False).str.replace(",", ".", regex=False)
    return pd.to_numeric(s, errors="coerce")

def load_dvf_chunked(dvf_path=raw_txt, out_fp=clean_fp):
    if os.path.exists(out_fp):
        return pd.read_parquet(out_fp)

    usecols = [
        "Date mutation","Nature mutation","Valeur fonciere",
        "Code postal","Commune","Code departement","Code commune",
        "Type local","Surface reelle bati","Nombre pieces principales"
    ]
    idf_prefix = ("75","77","78","91","92","93","94","95")
    chunksize = 200_000
    keep = []

    # Try UTF-8 first (file shows UTF-8 symptoms), then fallback CP1252
    tried = []
    for enc in ("utf-8", "cp1252"):
        try:
            reader = pd.read_csv(
                dvf_path, sep="|", dtype=str, usecols=usecols,
                chunksize=chunksize, low_memory=True, engine="c", encoding=enc
            )
            tried.append(enc)
            for i, chunk in enumerate(reader):
                # Filter Vente + IDF early
                chunk = chunk[chunk["Nature mutation"].fillna("").str.contains("Vente", case=False, na=False)]
                chunk = chunk[chunk["Code departement"].astype(str).str.startswith(idf_prefix)]
                if chunk.empty:
                    continue

                # Numeric conversions
                chunk["Valeur fonciere"]     = to_num_fr(chunk["Valeur fonciere"])
                chunk["Surface reelle bati"] = pd.to_numeric(chunk["Surface reelle bati"], errors="coerce")

                # Date parse — explicit dd/mm/yyyy
                dm = chunk["Date mutation"].astype(str).str.strip()
                dt = pd.to_datetime(dm, format="%d/%m/%Y", errors="coerce")

                # Small fallback for rare ISO rows (yyyy-mm-dd)
                m = dt.isna()
                if m.any():
                    dt.loc[m] = pd.to_datetime(dm.loc[m], errors="coerce", dayfirst=True)
                chunk["Date mutation"] = dt

                # Keep only plausible values
                chunk = chunk[(chunk["Surface reelle bati"] > 8) & (chunk["Valeur fonciere"] > 1000)]
                chunk["prix_m2"] = chunk["Valeur fonciere"] / chunk["Surface reelle bati"]
                chunk = chunk[chunk["prix_m2"].between(100, 30000)]

                # Build INSEE key (IDF => dept 2 + commune 3)
                chunk = chunk.rename(columns={
                    "Date mutation":"date_mutation",
                    "Valeur fonciere":"valeur_fonciere",
                    "Surface reelle bati":"surface_reelle_bati",
                    "Commune":"nom_commune",
                    "Code postal":"code_postal",
                    "Type local":"type_local",
                    "Code departement":"code_departement",
                    "Code commune":"code_commune_3"
                })
                chunk["code_departement"]   = chunk["code_departement"].astype(str).str.strip().str.zfill(2)
                chunk["code_commune_3"]     = chunk["code_commune_3"].astype(str).str.strip().str.zfill(3)
                chunk["code_commune_insee"] = chunk["code_departement"] + chunk["code_commune_3"]

                keep.append(chunk[[
                    "date_mutation","valeur_fonciere","surface_reelle_bati","prix_m2",
                    "nom_commune","code_postal","type_local","code_commune_insee"
                ]])

                if (i+1) % 5 == 0:
                    print(f"[{enc}] chunks: {i+1}  rows kept: ~{sum(len(k) for k in keep):,}")
            break
        except Exception as e:
            tried.append(f"{enc} (failed: {e.__class__.__name__})")
            continue

    if not keep:
        raise RuntimeError(f"DVF read yielded no rows. Tried encodings: {tried}")

    df = pd.concat(keep, ignore_index=True)
    df["annee"] = pd.to_datetime(df["date_mutation"], errors="coerce").dt.year
    df.to_parquet(out_fp, index=False)
    print(f"[OK] Saved → {out_fp}  rows={len(df):,}")
    return df

dvf = load_dvf_chunked()
print(dvf.shape, dvf["date_mutation"].isna().mean(), dvf["annee"].isna().mean())



Deleted: ..\data\clean\dvf_clean.parquet
[OK] Saved → c:\Users\KenziLali\OneDrive\iCloudDrive\Etudes\ING\ING4\S7\Data science\Projet-Data-science-Investissement-immobilier\data\clean\dvf_clean.parquet  rows=56,612
(56612, 9) 0.0 0.0


In [84]:
import re
import numpy as np
import pandas as pd

def parse_dates_robust(s: pd.Series) -> pd.Series:
    """Nettoyage + parsing multi-formats pour DVF date_mutation."""
    # 1) Normalise la chaîne (retire NBSP, trim, garde chiffres et /-)
    x = (s.astype(str)
           .str.replace("\u00A0"," ", regex=False)   # NBSP
           .str.strip()
           .str.replace(r"[^0-9/\-]", "", regex=True))

    out = pd.Series(pd.NaT, index=s.index, dtype="datetime64[ns]")

    # 2) yyyy-mm-dd
    m1 = x.str.match(r"^\d{4}-\d{2}-\d{2}$", na=False)
    if m1.any():
        out.loc[m1] = pd.to_datetime(x.loc[m1], format="%Y-%m-%d", errors="coerce")

    # 3) dd/mm/yyyy
    m2 = x.str.match(r"^\d{1,2}/\d{1,2}/\d{4}$", na=False)
    if m2.any():
        out.loc[m2] = pd.to_datetime(x.loc[m2], format="%d/%m/%Y", errors="coerce")

    # 4) dd/mm/yy → suppose 00–69 => 2000–2069, 70–99 => 1970–1999
    m3 = x.str.match(r"^\d{1,2}/\d{1,2}/\d{2}$", na=False)
    if m3.any():
        tmp = x.loc[m3].copy()
        # ajoute le siècle
        yy = tmp.str.extract(r"(\d{2})$")[0].astype(int)
        century = np.where(yy >= 70, "19", "20")
        # reconstruit en dd/mm/yyyy
        tmp_full = tmp.str.replace(r"/(\d{2})$", lambda m: f"/{century[tmp.index.get_loc(m.start())]}{m.group(1)}", regex=True)
        out.loc[m3] = pd.to_datetime(tmp_full, format="%d/%m/%Y", errors="coerce")

    # 5) Dernière passe permissive sur le reste (dayfirst=True)
    rem = out.isna()
    if rem.any():
        out.loc[rem] = pd.to_datetime(x.loc[rem], errors="coerce", dayfirst=True)

    return out

# --- applique sur ton df déjà chargé (depuis dvf_clean.parquet) ---
before = dvf.get("annee")
dvf["date_mutation"] = parse_dates_robust(dvf["date_mutation"])
dvf["annee"] = dvf["date_mutation"].dt.year

print("Taux de dates invalides après parse robuste :",
      dvf["date_mutation"].isna().mean().round(4))

# Diagnostique rapide si encore élevé
if dvf["date_mutation"].isna().mean() > 0.1:
    print("Exemples de valeurs non parsées :")
    print(dvf.loc[dvf["date_mutation"].isna(), "date_mutation"].astype(str).value_counts().head(10))

# Sauvegarde propre
dvf.to_parquet(clean_fp, index=False)
print("✅ Parquet regénéré :", clean_fp)


Taux de dates invalides après parse robuste : 0.0
✅ Parquet regénéré : c:\Users\KenziLali\OneDrive\iCloudDrive\Etudes\ING\ING4\S7\Data science\Projet-Data-science-Investissement-immobilier\data\clean\dvf_clean.parquet


In [85]:
import re
import pandas as pd

# 1) Retrouver la colonne date quel que soit son nom
name_map = {c.lower().strip(): c for c in dvf.columns}
date_candidates = ["date_mutation", "date mutation", "date_mut", "date"]
date_col = next((name_map[k] for k in date_candidates if k in name_map), None)

if date_col is None:
    # tentative heuristique: première colonne contenant 'date'
    date_col = next((c for c in dvf.columns if re.search(r"\bdate\b", c, flags=re.I)), None)

if date_col is None:
    raise KeyError(f"Aucune colonne date trouvée dans dvf. Colonnes: {list(dvf.columns)}")

# 2) (Re)calculer l'année de façon sûre
# Recalcule proprement l'année avec format français (jour en premier)
dvf["annee"] = pd.to_datetime(dvf["date_mutation"], errors="coerce", dayfirst=True).dt.year

# Vérifie la validité
nan_rate = dvf["annee"].isna().mean()
print(f"Taux de dates invalides après correction : {nan_rate:.1%}")

# Sauvegarde propre
dvf.to_parquet(clean_fp, index=False)
print("✅ Parquet mis à jour avec dates corrigées :", clean_fp)

# 3) Optionnel: si beaucoup de NaN d'année, signale le problème de parsing
nan_rate = dvf["annee"].isna().mean()
if nan_rate > 0.5:
    print(f"⚠️ Plus de 50% des dates sont invalides ({nan_rate:.0%}). Vérifie le format de '{date_col}'.")

# 4) Sauvegarder à nouveau le parquet avec 'annee' pour accélérer les prochains runs
from pathlib import Path
CLEAN_DIR = Path(CLEAN_DIR)
CLEAN_DIR.mkdir(parents=True, exist_ok=True)
clean_fp = CLEAN_DIR / "dvf_clean.parquet"
dvf.to_parquet(clean_fp, index=False)
print("✅ 'annee' ajoutée et parquet mis à jour →", clean_fp)



Taux de dates invalides après correction : 0.0%
✅ Parquet mis à jour avec dates corrigées : c:\Users\KenziLali\OneDrive\iCloudDrive\Etudes\ING\ING4\S7\Data science\Projet-Data-science-Investissement-immobilier\data\clean\dvf_clean.parquet
✅ 'annee' ajoutée et parquet mis à jour → c:\Users\KenziLali\OneDrive\iCloudDrive\Etudes\ING\ING4\S7\Data science\Projet-Data-science-Investissement-immobilier\data\clean\dvf_clean.parquet


In [86]:
display(dvf.head(5))
print(dvf["annee"].value_counts().sort_index().tail(10))
print(dvf["nom_commune"].value_counts().head(10))


,date_mutation,valeur_fonciere,surface_reelle_bati,prix_m2,nom_commune,code_postal,type_local,code_commune_insee,annee
0,2025-01-06,121000.0,69.0,1753.623188,COMBS LA VILLE,77380,Appartement,77122,2025
1,2025-01-03,246000.0,49.0,5020.408163,AVON,77210,Appartement,77014,2025
2,2025-01-08,318050.0,115.0,2765.652174,ROISSY-EN-BRIE,77680,Maison,77390,2025
3,2025-01-03,163000.0,42.0,3880.952381,ROISSY-EN-BRIE,77680,Appartement,77390,2025
4,2025-01-06,150000.0,93.0,1612.903226,ROISSY-EN-BRIE,77680,Maison,77390,2025


annee
2025    56612
Name: count, dtype: int64
nom_commune
PARIS 18                1302
PARIS 15                1171
PARIS 17                1077
PARIS 16                1028
PARIS 11                 911
PARIS 20                 859
PARIS 14                 703
PARIS 10                 694
BOULOGNE-BILLANCOURT     689
PARIS 12                 672
Name: count, dtype: int64


popo


In [87]:
import os, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntRangeSlider, FloatSlider, IntSlider, Checkbox, HBox, VBox, Output, HTML
from IPython.display import display, clear_output

warnings.filterwarnings("ignore")

CLEAN_DIR = os.path.abspath(os.path.join("..", "data", "clean"))
df = pd.read_parquet(os.path.join(CLEAN_DIR, "dvf_clean.parquet"))

# colonnes minimales attendues
needed = {"date_mutation","annee","valeur_fonciere","surface_reelle_bati","prix_m2","nom_commune"}
missing = needed - set(df.columns)
assert not missing, f"Colonnes manquantes: {missing}"

# filtres de base (par sécurité)
df = df.dropna(subset=["surface_reelle_bati","valeur_fonciere","prix_m2","nom_commune"]).copy()
df = df[(df["surface_reelle_bati"] > 8) & (df["valeur_fonciere"] > 1000) & (df["prix_m2"].between(100, 30000))]

len(df), df["annee"].min(), df["annee"].max()


(56612, np.int32(2025), np.int32(2025))

In [88]:
w_surface = IntRangeSlider(value=(20, 80), min=10, max=200, step=1,
                           description="Surface (m²)", continuous_update=False)

w_loyer = FloatSlider(value=22.0, min=5.0, max=45.0, step=0.5,
                      readout_format=".1f", description="Loyer €/m²", continuous_update=False)

w_topn = IntSlider(value=15, min=5, max=50, step=1, description="Top N", continuous_update=False)
w_out  = Checkbox(value=True, description="Filtrer outliers (IQR)")

out = Output()
display(VBox([HBox([w_surface, w_loyer]), HBox([w_topn, w_out]), out]))


In [89]:
def iqr_bounds(s, k=2.0):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    return q1 - k*iqr, q3 + k*iqr

def apply_outliers(d, use_iqr=True):
    if not use_iqr or len(d) < 50:
        return d
    lo, hi = iqr_bounds(d["prix_m2"], k=2.0)
    return d[d["prix_m2"].between(lo, hi)]

def filter_by_surface(d: pd.DataFrame, s_range):
    smin, smax = s_range
    return d[d["surface_reelle_bati"].between(smin, smax)]

def compute_yield(d: pd.DataFrame, loyer_m2: float) -> pd.DataFrame:
    res = d.copy()
    res["revenu_annuel"] = loyer_m2 * res["surface_reelle_bati"] * 12.0
    res["yield_brut"]    = res["revenu_annuel"] / res["valeur_fonciere"]
    return res

def render(_=None):
    with out:
        clear_output(wait=True)

        d = filter_by_surface(df, w_surface.value)
        d = apply_outliers(d, w_out.value)
        d = compute_yield(d, w_loyer.value)

        # Classement communes par rendement médian
        rank = (d.groupby("nom_commune", dropna=False)
                  .agg(nb=("prix_m2","count"),
                       prix_m2_med=("prix_m2","median"),
                       surf_med=("surface_reelle_bati","median"),
                       yield_brut_med=("yield_brut","median"),
                       yield_p90=("yield_brut", lambda s: s.quantile(0.90)))
                  .sort_values("yield_brut_med", ascending=False)
                  .head(w_topn.value)
                  .reset_index())

        display(HTML("<h4>Top communes (rendement brut médian)</h4>"))
        display(rank)

        # Histogramme (cap à 25% pour lisibilité)
        plt.figure()
        d["yield_brut"].dropna().clip(upper=0.25).plot(kind="hist", bins=40)
        plt.xlabel("Rendement brut")
        plt.ylabel("Nombre de ventes")
        plt.title("Distribution des rendements (Surface + Loyer €/m²)")
        plt.show()

w_surface.observe(render, "value")
w_loyer.observe(render, "value")
w_topn.observe(render, "value")
w_out.observe(render, "value")
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntRangeSlider, FloatSlider, Checkbox, IntSlider, Output, VBox, HBox, HTML
from IPython.display import display, clear_output

# --- Widgets ---
w_surface = IntRangeSlider(
    value=[15, 60],
    min=10, max=200, step=5,
    description='Surface (m²)',
    continuous_update=False,
    layout={'width': '400px'}
)

w_loyer = FloatSlider(
    value=25.0, min=10, max=60, step=0.5,
    description='Loyer €/m²/mois',
    continuous_update=False,
    layout={'width': '400px'}
)

w_topn = IntSlider(
    value=10, min=5, max=50, step=5,
    description='Top N',
    continuous_update=False,
    layout={'width': '400px'}
)

w_out = Checkbox(
    value=True,
    description='Exclure outliers (IQR)',
)

out = Output()

# --- Assemblage des contrôles ---
controls = VBox([
    HTML("<h3>Paramètres d’analyse</h3>"),
    HBox([w_surface, w_loyer]),
    HBox([w_topn, w_out])
])

display(controls, out)

render()


Output()

In [90]:
# ==== Dashboard Surface & Loyer (Widgets 5 et 6) ====
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ipywidgets as W
from IPython.display import display, clear_output, HTML

# --------- Helpers métriques ---------
def iqr_bounds(s, k=2.0):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    return q1 - k*iqr, q3 + k*iqr

def apply_outliers(d, use_iqr=True):
    if not use_iqr or len(d) < 50:
        return d
    lo, hi = iqr_bounds(d["prix_m2"], k=2.0)
    return d[d["prix_m2"].between(lo, hi)]

def filter_by_surface(d: pd.DataFrame, s_range):
    smin, smax = s_range
    return d[d["surface_reelle_bati"].between(smin, smax)]

def compute_yield(d: pd.DataFrame, loyer_m2: float) -> pd.DataFrame:
    res = d.copy()
    res["revenu_annuel"] = loyer_m2 * res["surface_reelle_bati"] * 12.0
    res["yield_brut"]    = res["revenu_annuel"] / res["valeur_fonciere"]
    return res

def kpis_html(d):
    nb = f"{len(d):,}".replace(",", " ")
    p50 = (d["prix_m2"].median() if len(d) else np.nan)
    y50 = (d["yield_brut"].median()*100 if len(d) else np.nan)
    return HTML(f"""
    <div style='display:flex;gap:28px;margin:8px 0 12px 0;font-family:system-ui,Segoe UI,Roboto,Arial'>
      <div><div style="opacity:.7">Nb ventes</div><div style="font-size:18px">{nb}</div></div>
      <div><div style="opacity:.7">Prix/m² médian</div><div style="font-size:18px">{p50:.0f} €</div></div>
      <div><div style="opacity:.7">Rendement médian</div><div style="font-size:18px">{y50:.2f} %</div></div>
    </div>""")

# --------- Widgets (5 & 6 + options utiles) ---------
w_surface = W.IntRangeSlider(value=[20, 80], min=10, max=200, step=1,
                             description='Surface (m²)', continuous_update=False, layout=W.Layout(width='360px'))
w_loyer   = W.FloatSlider(value=22.0, min=5.0, max=45.0, step=0.5,
                          readout_format=".1f", description='Loyer €/m²', continuous_update=False, layout=W.Layout(width='360px'))
w_topn    = W.IntSlider(value=15, min=5, max=50, step=1, description='Top N', continuous_update=False, layout=W.Layout(width='360px'))
w_out     = W.Checkbox(value=True, description='Exclure outliers (IQR)')

# (option) filtre par type_local si présent
type_opts = sorted(df["type_local"].dropna().unique()) if "type_local" in df.columns else []
w_type    = W.SelectMultiple(options=type_opts, value=tuple(type_opts) if type_opts else (),
                             description='Type', rows=min(6, len(type_opts))) if type_opts else None

# Actions
btn_reset = W.Button(description="Réinitialiser filtres")
btn_export = W.Button(description="Exporter le classement (CSV)")

# Zones d’affichage
out_table = W.Output()
out_plot1 = W.Output()
out_plot2 = W.Output()
out_kpi   = W.Output()

# --------- Rendu principal ---------
def render(_=None):
    with out_table:
        clear_output(wait=True)
    with out_plot1:
        clear_output(wait=True)
    with out_plot2:
        clear_output(wait=True)
    with out_kpi:
        clear_output(wait=True)

    # 1) filtrage surface + (option) type
    d = filter_by_surface(df, w_surface.value)
    if w_type and len(w_type.value) > 0:
        d = d[d["type_local"].isin(w_type.value)]

    # 2) outliers IQR
    d = apply_outliers(d, w_out.value)

    # 3) rendement
    d = compute_yield(d, w_loyer.value)

    # 4) KPIs
    with out_kpi:
        display(kpis_html(d))

    # 5) Classement communes
    rank = (d.groupby("nom_commune", dropna=False)
              .agg(nb=("prix_m2","count"),
                   prix_m2_med=("prix_m2","median"),
                   surf_med=("surface_reelle_bati","median"),
                   yield_brut_med=("yield_brut","median"),
                   yield_p90=("yield_brut", lambda s: s.quantile(0.90)))
              .sort_values("yield_brut_med", ascending=False)
              .head(w_topn.value)
              .reset_index())

    # 6) Tableau
    with out_table:
        display(HTML("<h4>Top communes (rendement brut médian)</h4>"))
        display(rank)

    # 7) Graphes (Matplotlib uniquement)
    # 7a) Histogramme des rendements
    with out_plot1:
        plt.figure(figsize=(6,4))
        d["yield_brut"].dropna().clip(upper=0.25).plot(kind="hist", bins=40)
        plt.xlabel("Rendement brut")
        plt.ylabel("Nombre de ventes")
        plt.title("Distribution des rendements (Surface & Loyer)")
        plt.show()

    # 7b) Barplot Top N par rendement médian
    with out_plot2:
        plt.figure(figsize=(7.5,4.5))
        x = rank["nom_commune"].astype(str)
        y = (rank["yield_brut_med"]*100).round(2)
        plt.bar(x, y)
        plt.xticks(rotation=60, ha="right")
        plt.ylabel("Rendement médian (%)")
        plt.title("Top communes par rendement médian")
        plt.tight_layout()
        plt.show()

    # 8) Stocker le dernier ranking pour export
    btn_export._last_rank = rank.copy()

# --------- Actions boutons ---------
def on_reset(_):
    w_surface.value = (20, 80)
    w_loyer.value   = 22.0
    w_topn.value    = 15
    w_out.value     = True
    if w_type:
        w_type.value = tuple(type_opts)
    render()

def on_export(_):
    rank = getattr(btn_export, "_last_rank", None)
    if rank is None or rank.empty:
        with out_table:
            display(HTML("<i>Aucun classement à exporter.</i>"))
        return
    # Export dans data/clean
    out_csv = os.path.join(CLEAN_DIR, "classement_communes_surface_loyer.csv")
    rank.to_csv(out_csv, index=False, encoding="utf-8")
    with out_table:
        display(HTML(f"<b>Exporté :</b> {out_csv}"))

btn_reset.on_click(on_reset)
btn_export.on_click(on_export)

# --------- Layout / Interface ---------
controls_left = [
    HTML("<h3 style='margin:0 0 6px 0'>Filtres</h3>"),
    w_surface, w_loyer, w_topn, w_out
]
if w_type:
    controls_left.insert(2, w_type)

box_left = W.VBox(controls_left + [W.HBox([btn_reset, btn_export])],
                  layout=W.Layout(width="400px"))

box_right = W.VBox([
    out_kpi,
    out_table,
    W.HBox([out_plot1, out_plot2])
])

ui = W.HBox([box_left, box_right], layout=W.Layout(align_items="flex-start"))
display(ui)

# Observers
w_surface.observe(render, "value")
w_loyer.observe(render, "value")
w_topn.observe(render, "value")
w_out.observe(render, "value")
if w_type:
    w_type.observe(render, "value")

# Premier rendu
render()


TraitError: The 'children' trait of a VBox instance contains an Instance of a TypedTuple which expected a Widget, not the HTML <IPython.core.display.HTML object>.